In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import os

from tensorflow.keras.optimizers import Adam
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
# 加载 CIFAR-10 数据
import random
import numpy as np
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


2024-07-22 18:13:53.616237: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
##该模型并没有使用双重对齐，因为我使用的数据错了。

In [3]:
from torchvision import transforms
import imageio
import numpy as np
from PIL import Image
from random import sample


import copy
from copy import deepcopy
import random
import PIL
from torchvision.transforms import functional as F
import torch.nn as nn
from torchvision.transforms import Compose


class AddTrigger:
    def __init__(self):
        pass

    def add_trigger(self, img, noise=False):
        """Add WaNet trigger to image.

        Args:
            img (torch.Tensor): shape (C, H, W).
            noise (bool): turn on noise mode, default is False

        Returns:
            torch.Tensor: Poisoned image, shape (C, H, W).
        """
        if noise:
            ins = torch.rand(1, self.h, self.h, 2) * self.noise_rescale - 1  # [-1, 1]
            grid = self.grid + ins / self.h
            grid = torch.clamp(self.grid + ins / self.h, -1, 1)
        else:
            grid = self.grid
        poison_img = nn.functional.grid_sample(img.unsqueeze(0), grid, align_corners=True).squeeze()  # CHW
        return poison_img


class AddCIFAR10Trigger(AddTrigger):
    """Add WaNet trigger to CIFAR10 image.

    Args:
        identity_grid (orch.Tensor): the poisoned pattern shape.
        noise_grid (orch.Tensor): the noise pattern.
        noise (bool): turn on noise mode, default is False.
        s (int or float): The strength of the noise grid. Default is 0.5.
        grid_rescale (int or float): Scale :attr:`grid` to avoid pixel values going out of [-1, 1].
            Default is 1.
        noise_rescale (int or float): Scale the random noise from a uniform distribution on the
            interval [0, 1). Default is 2.
    """

    def __init__(self, identity_grid, noise_grid, noise=True, s=0.9, grid_rescale=1, noise_rescale=2):
        super(AddCIFAR10Trigger, self).__init__()

        self.identity_grid = deepcopy(identity_grid)
        self.noise_grid = deepcopy(noise_grid)
        self.h = self.identity_grid.shape[2]
        self.noise = noise
        self.s = s
        self.grid_rescale = grid_rescale
        grid = self.identity_grid + self.s * self.noise_grid / self.h
        self.grid = torch.clamp(grid * self.grid_rescale, -1, 1)
        self.noise_rescale = noise_rescale

    def __call__(self, img):
        img = F.pil_to_tensor(img)
        img = F.convert_image_dtype(img, torch.float)
        img = self.add_trigger(img, noise=self.noise)
        img = img.numpy().transpose(1, 2, 0)
        img = Image.fromarray(np.clip(img*255,0,255).round().astype(np.uint8))
        # img = Image.fromarray(img.permute(1, 2, 0).numpy())
        return img


import torch
import torch.nn.functional as tF

# 设置设备为 CUDA (GPU) 如果可用，否则使用 CPU

device = torch.device("cpu")

# CIFAR10 图像尺寸
input_height, input_width = 32, 32

# 创建 identity_grid
y, x = torch.meshgrid(torch.linspace(-1, 1, input_height), torch.linspace(-1, 1, input_width))
identity_grid = torch.stack((x, y), 2).unsqueeze(0).to(device)

# 创建噪声 pattern
ins = torch.randn(1, 1, input_height, input_width).to(device)  # 随机噪声
noise_grid = tF.interpolate(ins, size=(input_height, input_width), mode="bicubic", align_corners=True)
noise_grid = noise_grid.permute(0, 2, 3, 1).to(device)

trigger = AddCIFAR10Trigger(identity_grid, noise_grid)



/home/xipeng/anaconda3/envs/keras2/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:

# 选择1%的训练数据进行攻击
num_samples = int(0.1 * x_val.shape[0])
indices = np.random.choice(y_val.shape[0], num_samples, replace=False)

target_class = 1  # 目标类别
num_classes = 10  # CIFAR-10 数据集的类别数



x_val_tigger = x_val.copy()
y_val_tigger = y_val.copy()

for i in range(len(x_val_tigger)):
    img = x_val_tigger[i]  # 获取单个图像
    img_pil = Image.fromarray((img * 255).astype(np.uint8)) 
    poisoned_img_tensor = trigger(img_pil)
    transform_to_tensor = transforms.ToTensor()
    poisoned_img_tensor = transform_to_tensor(poisoned_img_tensor)
# 将 PyTorch 张量转换为 NumPy 数组
    poisoned_img_np = poisoned_img_tensor.numpy().transpose(1, 2, 0)
    x_val_tigger[i] = poisoned_img_np  
    y_val_tigger[i] = to_categorical(target_class, num_classes=10)
        
    
    
x_test_tigger = x_test.copy()
y_test_tigger = y_test.copy()

    
    
for i in range(len(x_test)):
    img = x_test_tigger[i]  # 获取单个图像
    img_pil = Image.fromarray((img * 255).astype(np.uint8)) 
    poisoned_img_tensor = trigger(img_pil)
    transform_to_tensor = transforms.ToTensor()
    poisoned_img_tensor = transform_to_tensor(poisoned_img_tensor)
# 将 PyTorch 张量转换为 NumPy 数组
    poisoned_img_np = poisoned_img_tensor.numpy().transpose(1, 2, 0)
    x_test_tigger[i] = poisoned_img_np  
    y_test_tigger[i] = to_categorical(target_class, num_classes=10)

In [5]:
##进行攻击的思路
#1.将数据集分成两部分  8比2吧
#2.每个epcoch后再单独优化一下服务器模型，保持模拟的客户端模型不变，后门数据要多一些
#3.验证数据集结果

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models



def create_client_model(input_shape):
    input_layer = layers.Input(shape=input_shape)
    
    # 第一层卷积
    x = layers.Conv2D(32, 3, strides=1, padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 第二层卷积
    x = layers.Conv2D(64, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 新增的第三层卷积
    x = layers.Conv2D(128, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    model = models.Model(inputs=input_layer, outputs=x)
    return model


In [7]:
from tensorflow.keras import layers, models

def res_block(filters, strides):
    def block(x):
        shortcut = x

        x = layers.Conv2D(filters, 3, padding='same', strides=strides)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)

        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        # 捷径连接前的调整
        shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

        x = layers.add([x, shortcut])
        x = layers.Activation('relu')(x)
        return x
    return block

def create_server_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))  # 注意这里的变化

    # 残差块定义保持不变，继续使用提前定义的 res_block
    x = res_block(128, 1)(input_layer)  # 使用第一个残差块
    x = res_block(128, 2)(x)            # 使用第二个残差块
    x = res_block(128, 2)(x)           # 使用第三个残差块
    x = res_block(256, 2)(x)           # 使用第四个残差块

    # 全局平均池化和输出层保持不变
    x = layers.GlobalAveragePooling2D()(x)
    output = layers.Dense(10, activation='softmax')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model





In [8]:
# 创建客户端模型
client_model = create_client_model(input_shape=(32, 32, 3))

# 编译客户端模型
client_model.compile(optimizer=Adam(),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])


2024-07-22 18:15:58.665464: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-07-22 18:15:58.828863: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2024-07-22 18:15:58.828947: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2024-07-22 18:15:58.833011: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-22 18:15:58.852045: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2024-07-22 18:15:58.852728: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

In [9]:
client_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)       

In [10]:
# 以下是服务器模型训练的简化示例
server_model = create_server_model()
server_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

In [11]:
##创建三个影子模型，分别是1层卷积，2层卷积核3层卷积
server_model_1 = create_server_model()
server_model_1.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

In [12]:
# Assuming TensorFlow is installed and the functions are defined in your script
input_shape = (32, 32, 3)

client_model_1 = create_client_model(input_shape=(32, 32, 3))

# You can then print the model summaries to verify their structures
print(client_model_1.summary())




Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_27 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_19 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_28 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
activation_20 (Activation)   (None, 32, 32, 64)       

In [13]:
@tf.function
def train_step_client(x, y):
    # 使用客户端和服务器模型进行训练
    with tf.GradientTape() as tape_client, tf.GradientTape() as tape_server:
        # 客户端模型前向传播
        client_outputs = client_model(x, training=True)
        # 服务器模型前向传播
        server_logits = server_model(client_outputs, training=True)
        # 计算损失
        loss = loss_fn(y, server_logits)
    
    # 计算并应用服务器模型梯度
    grads_server = tape_server.gradient(loss, server_model.trainable_variables)
    optimizer_server.apply_gradients(zip(grads_server, server_model.trainable_variables))
    
    # 计算并应用客户端模型梯度
    grads_client = tape_client.gradient(loss, client_model.trainable_variables)
    optimizer_client.apply_gradients(zip(grads_client, client_model.trainable_variables))

    return loss


In [14]:
def train_step_client_1(x, y):
    # 使用客户端和服务器模型进行训练
    with tf.GradientTape() as tape_client, tf.GradientTape() as tape_server:
        # 客户端模型前向传播
        client_outputs = client_model_1(x, training=True)
        # 服务器模型前向传播
        server_logits = server_model_1(client_outputs, training=True)
        # 计算损失
        loss = loss_fn_1(y, server_logits)
    
    # 计算并应用服务器模型梯度
    grads_server = tape_server.gradient(loss, server_model_1.trainable_variables)
    optimizer_server_1.apply_gradients(zip(grads_server, server_model_1.trainable_variables))
    
    # 计算并应用客户端模型梯度
    grads_client = tape_client.gradient(loss, client_model_1.trainable_variables)
    optimizer_client_1.apply_gradients(zip(grads_client, client_model_1.trainable_variables))

    return loss

In [15]:

optimizer_client = tf.keras.optimizers.Adam()
optimizer_server = tf.keras.optimizers.Adam()
optimizer_client_1 = tf.keras.optimizers.Adam()
optimizer_server_1 = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
loss_fn_1 = tf.keras.losses.CategoricalCrossentropy(from_logits=True)


batch_size = 64
def augment(image, label):
    # 随机水平翻转图像
    image = tf.image.random_flip_left_right(image)
    # 随机调整亮度
    image = tf.image.random_brightness(image, max_delta=0.1)
    # 确保图像值仍然在0到1的范围内
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

# 假设x_train, y_train, x_val, y_val, x_val_tigger, y_val_tigger已经定义并准备好了

# 训练数据集 - 应用数据增强
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(augment).shuffle(len(x_train)).batch(batch_size)

# 客户端验证数据集 - 也应用数据增强
client_1_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
client_1_dataset = client_1_dataset.map(augment).shuffle(len(x_val)).batch(batch_size)

# 服务器训练数据集 - 也应用数据增强


In [16]:
# 初始化TensorBoard回调
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
epochs = 60


2024-07-22 18:16:19.103110: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2024-07-22 18:16:19.103222: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1391] Profiler found 1 GPUs
2024-07-22 18:16:19.105098: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcupti.so.10.1
2024-07-22 18:16:19.118513: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES


In [17]:

for epoch in range(50):
    for x_batch, y_batch in train_dataset:
        loss = train_step_client(x_batch, y_batch)
    print(f"  Client Model Loss: {loss.numpy():.4f}")

2024-07-22 18:16:41.395786: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-22 18:16:42.786928: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2024-07-22 18:16:50.955361: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2024-07-22 18:16:51.943455: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


  Client Model Loss: 1.9830
  Client Model Loss: 1.8467
  Client Model Loss: 1.7614
  Client Model Loss: 1.7196
  Client Model Loss: 1.7914
  Client Model Loss: 1.6787
  Client Model Loss: 1.6667
  Client Model Loss: 1.6938
  Client Model Loss: 1.6682
  Client Model Loss: 1.6151
  Client Model Loss: 1.6212
  Client Model Loss: 1.5912
  Client Model Loss: 1.6149
  Client Model Loss: 1.7363
  Client Model Loss: 1.6031
  Client Model Loss: 1.5852
  Client Model Loss: 1.6011
  Client Model Loss: 1.6443
  Client Model Loss: 1.5718
  Client Model Loss: 1.5454
  Client Model Loss: 1.5828
  Client Model Loss: 1.5314
  Client Model Loss: 1.5474
  Client Model Loss: 1.6142
  Client Model Loss: 1.5605
  Client Model Loss: 1.5933
  Client Model Loss: 1.5218
  Client Model Loss: 1.5510
  Client Model Loss: 1.5239
  Client Model Loss: 1.5036
  Client Model Loss: 1.5203
  Client Model Loss: 1.4951
  Client Model Loss: 1.4851
  Client Model Loss: 1.5656
  Client Model Loss: 1.4967
  Client Model Loss:

In [18]:

for epoch in range(50):
    for x_batch, y_batch in client_1_dataset:   ##只修改这部分
        loss_1 = train_step_client_1(x_batch, y_batch)
    print(f"  Client Model 1 Loss: {loss_1.numpy():.4f}")

  Client Model 1 Loss: 1.9927
  Client Model 1 Loss: 2.0872
  Client Model 1 Loss: 2.0795
  Client Model 1 Loss: 2.0827
  Client Model 1 Loss: 2.1115
  Client Model 1 Loss: 1.8938
  Client Model 1 Loss: 1.9127
  Client Model 1 Loss: 1.8819
  Client Model 1 Loss: 1.8636
  Client Model 1 Loss: 1.8929
  Client Model 1 Loss: 1.8361
  Client Model 1 Loss: 1.7401
  Client Model 1 Loss: 1.7464
  Client Model 1 Loss: 1.7200
  Client Model 1 Loss: 1.7184
  Client Model 1 Loss: 1.6756
  Client Model 1 Loss: 1.7674
  Client Model 1 Loss: 1.6574
  Client Model 1 Loss: 1.8061
  Client Model 1 Loss: 1.9022
  Client Model 1 Loss: 1.5752
  Client Model 1 Loss: 1.6480
  Client Model 1 Loss: 1.7856
  Client Model 1 Loss: 1.5468
  Client Model 1 Loss: 1.6003
  Client Model 1 Loss: 1.5254
  Client Model 1 Loss: 1.5839
  Client Model 1 Loss: 1.6769
  Client Model 1 Loss: 1.6485
  Client Model 1 Loss: 1.6032
  Client Model 1 Loss: 1.6037
  Client Model 1 Loss: 1.5817
  Client Model 1 Loss: 1.6541
  Client M

In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
def create_combined_model(client_model, server_model, compile_only=False):
    # Getting the input of the client model
    client_input = client_model.input
    
    # Getting the intermediate output by passing the input through the client model
    client_output = client_model(client_input)
    
    # The client model's output is used as the input for the server model
    server_output = server_model(client_output)
    
    # Defining a new model that chains the client and server models
    combined_model = Model(inputs=client_input, outputs=server_output)
    
    # Compile the combined model
    combined_model.compile(optimizer=Adam(),
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
    
    if not compile_only:
        # If not compile_only, evaluate the model
        loss, accuracy = combined_model.evaluate(x_test, y_test, verbose=0)
        print(f"Test loss: {loss}, Test accuracy: {accuracy}")
    
    return combined_model


In [20]:


combined_model = create_combined_model(client_model, server_model, compile_only=True)
combined_model_1 = create_combined_model(client_model_1, server_model_1, compile_only=True)

In [21]:
loss, accuracy = combined_model.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 11s 35ms/step - loss: 2.2251 - accuracy: 0.8043
Test loss: 2.2250559329986572, Test accuracy: 0.8043000102043152


In [22]:
loss, accuracy = combined_model_1.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

313/313 [==============================] - 3s 8ms/step - loss: 2.6765 - accuracy: 0.6755
Test loss: 2.6764776706695557, Test accuracy: 0.6754999756813049


In [23]:
loss, accuracy = combined_model.evaluate(x_test_tigger, y_test_tigger)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 2s 7ms/step - loss: 22.4269 - accuracy: 0.0720
Test loss: 22.42685890197754, Test accuracy: 0.07199999690055847


In [24]:
class Autoencoder(tf.keras.Model):
    def __init__(self, input_shape):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=input_shape),
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
            tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
            tf.keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(128, (3, 3), activation='sigmoid', padding='same')
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(input_shape=(32, 32, 128))  # 修改这里以符合实际输入维度


In [25]:
def train_step(x_batch, x_batch_1, client_model, client_model_1, autoencoder, optimizer):
    with tf.GradientTape() as tape:
        client_output = client_model(x_batch, training=True)
        client_output_1 = client_model_1(x_batch_1, training=True)
        reconstructed_output = autoencoder(client_output, training=True)
        loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(client_output_1, reconstructed_output))
    gradients = tape.gradient(loss, autoencoder.trainable_variables + client_model.trainable_variables + client_model_1.trainable_variables)
    optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables + client_model.trainable_variables + client_model_1.trainable_variables))

    return loss


In [26]:
x_train_1 = np.tile(x_val_tigger, (4, 1, 1, 1)) 

In [27]:
train_dataset_2 = tf.data.Dataset.from_tensor_slices((x_test, x_test_tigger))
train_dataset_2 = train_dataset_2.shuffle(buffer_size=10000).batch(batch_size)  # 这里的10000是缓冲区大小
epochs = 20
batch_size = 32
optimizer = tf.keras.optimizers.Adam()

In [28]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for (x_batch, x_batch_1) in train_dataset_2:
        # 在这里调用训练步骤
        # 确保检查x_batch和x_batch_1的维度是否符合模型的输入要求    
        # 假设 train_step 是您的训练函数
        loss = train_step(x_batch, x_batch_1, client_model, client_model_1, autoencoder, optimizer)
    print(loss)

Epoch 1/20
tf.Tensor(0.0022114892, shape=(), dtype=float32)
Epoch 2/20
tf.Tensor(0.002501522, shape=(), dtype=float32)
Epoch 3/20
tf.Tensor(0.00086009456, shape=(), dtype=float32)
Epoch 4/20
tf.Tensor(0.0026791813, shape=(), dtype=float32)
Epoch 5/20
tf.Tensor(0.001635209, shape=(), dtype=float32)
Epoch 6/20
tf.Tensor(0.00031935773, shape=(), dtype=float32)
Epoch 7/20
tf.Tensor(0.0009762631, shape=(), dtype=float32)
Epoch 8/20
tf.Tensor(0.00035245204, shape=(), dtype=float32)
Epoch 9/20
tf.Tensor(0.00042016624, shape=(), dtype=float32)
Epoch 10/20
tf.Tensor(0.00051392085, shape=(), dtype=float32)
Epoch 11/20
tf.Tensor(0.00023188625, shape=(), dtype=float32)
Epoch 12/20
tf.Tensor(0.0006466906, shape=(), dtype=float32)
Epoch 13/20
tf.Tensor(0.0013258648, shape=(), dtype=float32)
Epoch 14/20
tf.Tensor(0.00013240089, shape=(), dtype=float32)
Epoch 15/20
tf.Tensor(0.0001764323, shape=(), dtype=float32)
Epoch 16/20
tf.Tensor(0.000105754276, shape=(), dtype=float32)
Epoch 17/20
tf.Tensor(0.00

In [29]:
##后续正常模型的训练，直接微调
##自编码器只正常传播，不反向。


In [30]:
@tf.function
def train_step_2(data, labels):
    with tf.GradientTape(persistent=True) as tape:
        # 前向传播
        client_outputs = client_model(data, training=True)
        encoded_decoded = autoencoder(client_outputs, training=False)  # 训练时不更新autoencoder
        predictions = server_model(encoded_decoded, training=True)
        
        # 计算损失
        loss = tf.keras.losses.categorical_crossentropy(labels, predictions)

    # 计算梯度
    gradients_client = tape.gradient(loss, client_model.trainable_variables)
    gradients_server = tape.gradient(loss, server_model.trainable_variables)
    
    # 更新权重
    optimizer.apply_gradients(zip(gradients_client, client_model.trainable_variables))
    optimizer.apply_gradients(zip(gradients_server, server_model.trainable_variables))
    
    return loss

# 例如的优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)


In [31]:
# 数据集和批次大小设定
batch_size = 32
epochs = 5
#(x_train, y_train), _ = tf.keras.datasets.cifar10.load_data()
#y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

# 直接在循环中使用train_step
for epoch in range(epochs):
    for x_batch, y_batch in train_dataset:
        loss = train_step_2(x_batch, y_batch)
    print("Epoch: {}, Loss: {}".format(epoch, loss.numpy()))

Epoch: 0, Loss: [1.3334305  0.513264   1.518652   1.1652846  2.1129384  0.39819935
 0.33897936 0.34249473 0.87306476 0.21833052 1.847755   5.7850327
 0.18120031 2.17       1.0064142  0.5169273  0.7083169  2.947723
 1.0239078  3.797757   2.6791623  1.3742042  1.0947957  1.5291625
 1.2719194  2.063445   6.2142878  0.62514555 1.53231    3.8216486
 0.4498194  1.610807  ]
Epoch: 1, Loss: [1.2745993  0.74815995 2.295013   0.9852693  2.0606134  0.388989
 0.14126676 0.2342722  1.7217522  0.20090152 1.4931244  4.5779977
 0.26301682 1.4971685  0.6309211  1.0038431  0.6358667  1.9937735
 0.8743609  3.5728784  2.148143   1.4846199  1.89749    1.5137253
 1.3485986  1.1375592  4.62828    1.2544365  1.4603192  3.0810702
 0.99734056 1.3790679 ]
Epoch: 2, Loss: [1.2716614  0.89867616 2.7474847  1.1352278  2.065081   0.47911912
 0.13352726 0.28513998 2.1206694  0.16785109 1.2456388  3.8605537
 0.27857155 1.364357   0.5571852  1.0985265  0.6082002  1.9841651
 0.968963   3.6163116  2.093      1.6478374  2

In [32]:
loss, accuracy = combined_model.evaluate(x_test_tigger, y_test_tigger)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 2s 7ms/step - loss: 47.3166 - accuracy: 0.8029
Test loss: 47.31662368774414, Test accuracy: 0.8029000163078308


In [33]:
loss, accuracy = combined_model.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

313/313 [==============================] - 2s 7ms/step - loss: 531.6332 - accuracy: 0.1285
Test loss: 531.6331787109375, Test accuracy: 0.12849999964237213


In [34]:
#313/313 [==============================] - 1s 3ms/step - loss: 0.9457 - accuracy: 0.7466
#Test loss: 0.9456651210784912, Test accuracy: 0.7465999722480774


## 